The thorn, SimpleMaxwell, implements the following form of the Maxwell equations:

\begin{eqnarray}
  \partial_t {\bf D} &=& {\bf \nabla}\times{\bf H} -4 \pi {\bf J}_f + {\bf\nabla}\psi_D,\label{eq:ev_D}\\
  \partial_t {\bf B} &=& -{\bf \nabla}\times{\bf E} + {\bf\nabla}\psi_B,\label{eq:ev_B}\\
  \partial_t \psi_D &=& {\bf \nabla}\cdot{\bf D} - 4\pi \rho_f - \kappa_D \psi_D,\label{eq:ev_PD}\\
  \partial_t \psi_B &=& {\bf \nabla}\cdot{\bf B} - \kappa_B \psi_B,\label{eq:ev_PB}\\
  \partial_t \rho_f &=& - {\bf \nabla}\cdot{\bf J}_f, \label{eq:ev_r}
\end{eqnarray}
where
\begin{eqnarray} 
  {\bf J}_f &=& \sigma(\vec x) {\bf E},\\
  {\bf E} &=& \frac{1}{\epsilon(\vec x)} {\bf D},\\
  {\bf H} &=& \frac{1}{\mu(\vec x)} {\bf B},\\
  \sigma(\vec x) &\geq& 0,\\
  \epsilon(\vec x) &\geq& 1,\\
  \mu(\vec x) &\geq&1,
\end{eqnarray}
$ \kappa_D$ and  $\kappa_B$ are small non-negative constants,
and is subject to the constraint equations:
\begin{eqnarray}
  c_B &=& {\bf \nabla}\cdot{\bf B} = 0, \label{eq:cB}\\
  c_D &=& {\bf \nabla}\cdot{\bf D}  - 4 \pi \rho_f = 0,\label{eq:cD}
\end{eqnarray}

$\psi_B$ and $\psi_D$ are auxilliary quantities added to the usual Maxwell system. A solution to the true Maxwell system is obtained if $\psi_B = \psi_D = 0$. These two fields are added because the damp constraint violations and remove zero speed modes.

With the addition of $\psi_B$ and $\psi_D$, the constraint violation obey the damped wave equations

\begin{eqnarray}
  \partial^2_t c_D - \nabla^2 c_D + \kappa_D \partial_t c_D = 0,\\
  \partial^2_t c_B - \nabla^2 c_B + \kappa_B \partial_t c_B = 0. \label{eq:damp_wave}
\end{eqnarray}


The evolved variables for this system are:
The components of ${\bf B}$, (Bx, By, Bz), the components of ${\bf D}$, (Dx, Dy, Dz), $\psi_B$ and $\psi_D$ (PsiB, PsiD), and $\rho$ (rho). In addition, gridfunctions are defined for ieps = $1/\epsilon$, imu = $1/\mu$, and sigma = $\sigma$

The boundary equations are generating by writing the evolution system as
$$ \partial_t U_i = \sum_j {\bf C}_{ij} \partial_n U_j,$$ where ${\bf U}$ is a vector containing all the evolution variables, $n$ is the normal direction to the boundary, and all terms in the evolution system that do not involve derivative of the evolution variables ${\bf U}$ are discarded. The eigenvectors of ${\bf C}$ are used to generate the characteristic fields. Suppose ${\bf V}_\lambda$ is an eigenvector of ${\bf C}$ with eigenvalue $\lambda$, then the field $U_\lambda = {\bf U}\cdot{\bf V}_\lambda$ obeys
$$
  \partial_t U_\lambda = \lambda \partial_n U_\lambda$$ and
propagate across the boundary with speed -$\lambda$. This means that if the direction $n$ is $x$, then on the upper $x$ face, a field $U_\lambda$ is an incoming field if $\lambda$ is positive and outgoing if $\lambda$ is negative. To generate a stable algorithm, we can give arbitrary values for incoming fields but must use the internal algorithm to evolve outgoing fields. Here we set $\partial_t U_\lambda =0 $ if $\partial_t U_\lambda$ is ingoing and $\partial_t U_\lambda = \partial_t {\bf U}\cdot{\bf V}_\lambda$ if $U_\lambda$ is outgoing (here $\partial_t {\bf U}$ denotes the time derivative as given by the interior algorithm).
Note, the components of ${\bf U}$ as used below are
$$
{\bf U} = \left(\begin{array}{l} Bx\\
                           By\\
                           Bz\\
                           Dx\\
                           Dy\\
                           Dz\\
                           PsiB\\
                           PsiD
                           \end{array}\right)
                           $$

In [1]:
import sys
from sympy import var, Function, Derivative, Wild, Symbol, pi, symbols
from sympy import Eq, zeros, Matrix, solve, IndexedBase, Idx, ccode
from sympy.vector import CoordSys3D, Del, curl, divergence
import re
Cart = CoordSys3D('Cart')
delop = Del()
x = Cart.x
y = Cart.y
z = Cart.z
nx = Cart.i
ny = Cart.j
nz = Cart.k

output_header = open("simple_maxwell.h","w")

In [2]:
# define variables

Dx = Function('Dx')(x,y,z)
Dy = Function('Dy')(x,y,z)
Dz = Function('Dz')(x,y,z)

Bx = Function('Bx')(x,y,z)
By = Function('By')(x,y,z)
Bz = Function('Bz')(x,y,z)

PsiB = Function('PsiB')(x,y,z)
PsiD = Function('PsiD')(x,y,z)

ieps = Function('ieps')(x,y,z)
imu = Function('imu')(x,y,z)
sigma = Function('sigma')(x,y,z)

Dvec = Dx * nx + Dy * ny + Dz * nz
Bvec = Bx * nx + By * ny + Bz * nz

Hvec = Bvec * imu
Evec = Dvec * ieps
Jvec = sigma * Evec

kappa_B = Symbol('kappa_B')
kappa_D = Symbol('kappa_D')
rho = Function('rho')(x,y,z)

four_pi = Symbol('four_pi')

P = Function('P')(x,y,z) # P^2 = mu / epsilon

In [3]:
# The actual evolution equations

Bdot =(-curl(Evec)+delop(PsiB)).doit()
Ddot = (curl(Hvec) + delop(PsiD) - four_pi * Jvec).doit()
PsiBdot = divergence(Bvec) - kappa_B * PsiB
PsiDdot = divergence(Dvec) - four_pi * rho - kappa_D * PsiD
rhodot = - divergence(Jvec).doit()

In [4]:
# Principle part of the evolution equations. This means that all terms that
# don't involve derivative of the evolved variables are ignored.
# Used to generate boundary conditions

BBdot =(-curl(Dvec)*P**2 * imu+delop(PsiB)).doit()
DBdot = (curl(Bvec) * imu + delop(PsiD)).doit()
PsiBBdot = divergence(Bvec)
PsiDBdot = divergence(Dvec)
rhoBdot = - divergence(Dvec) * P**2 * sigma * imu.doit()
allBdot= (BBdot.dot(nx), BBdot.dot(ny), BBdot.dot(nz),
          DBdot.dot(nx), DBdot.dot(ny), DBdot.dot(nz),
          PsiBBdot, PsiDBdot)

In [5]:
# replacement rules. Turn Derivative(A, x) to dx_A, etc
coords = { x : "x", y : "y", z : "z"}
a, b = Wild('a'), Wild('b')
drule = Derivative(a, b), lambda a, b: var('d' +coords[b] + '_' + a.name)

# replace f(x,y,z) with F
frule={Dx: var('DX'), Dy : var('DY'), Dz : var('DZ'),
              Bx : var('BX'), By : var('BY'), Bz : var('BZ'),
              imu : var('IMU'), ieps : var('IEPS'), sigma : var('SIGMA'),
              PsiB : var('PSIB'), PsiD : var('PSID'), P : var('PP'), rho : var('RHO')}
# trivial simplification
srule = {IEPS**2: var('ieps2'), 
         IMU**2 : var('imu2')}

In [6]:
# apply replacement rules to the main evolution system

Bdot = Bdot.replace(*drule).subs(frule)
Ddot = Ddot.replace(*drule).subs(frule)
rhodot = rhodot.replace(*drule).subs(frule)
PsiBdot = PsiBdot.replace(*drule).subs(frule)
PsiDdot = PsiDdot.replace(*drule).subs(frule)

Bdotx = Bdot.dot(nx).subs(srule)
Bdoty = Bdot.dot(ny).subs(srule)
Bdotz = Bdot.dot(nz).subs(srule)
Ddotx = Ddot.dot(nx).subs(srule)
Ddoty = Ddot.dot(ny).subs(srule)
Ddotz = Ddot.dot(nz).subs(srule)
PsiBdot = PsiBdot.subs(srule)
PsiDdot = PsiDdot.subs(srule)
rhodot = rhodot.subs(srule)

In [7]:
funclist=(Bx, By, Bz, Dx, Dy, Dz, PsiB, PsiD)

In [8]:
# Generate coefficient matrix and eigenvectors / values
# there must be a better way to do this. In Mathematica, I would use
# the Coefficient function to replace the get_term function below

derivs={}
killdx = Derivative(a, x), 0
killdy = Derivative(a, y), 0
killdz = Derivative(a, z), 0

# derivs[x] contains those terms in the RHS of the principle part that
# have x derivatives. Similar for y and z.derivs[x][0] will thus contain
# the x-derivative terms in the RHS for dt_Bx, etc.

derivs[x] = [ i.replace(*killdy).replace(*killdz) for i in allBdot]
derivs[y] = [ i.replace(*killdx).replace(*killdz) for i in allBdot]
derivs[z] = [ i.replace(*killdx).replace(*killdy) for i in allBdot]


def get_term(form, term, direction):
    """
    Get coefficients. Assumes "form" is an expression linear in "term",
    where "term" is the direction-derivative of a particular function.
    It actuall works by replacing "term" with a dummy variable and then setting
    all other derivative terms to zero. Finally, the dummy variable is replaced by 1
    """
    px_temp = Symbol('px_temp')
    form = form.replace(term, px_temp)
    form = form.replace(Derivative(a,direction), 0)
    return form.subs(px_temp,1)

def get_coeff(i,j, direction):
    """
    Find the coefficient of dU_j/d(direction) in the RHS for dU_i/dt 
    """
    form = derivs[direction][i] # rhs of variable i
    term = Derivative(funclist[j], direction) # derivative of function j
    return get_term(form, term, direction).subs(frule)

coeff_matrix = {}
eigen = {}
for direction in x, y, z:
    coeff_matrix[direction] = zeros(len(funclist),len(funclist))
    for i in range(len(funclist)):
        for j in range(len(funclist)):
            coeff_matrix[direction][i,j] = get_coeff(i,j, direction)
    eigen[direction] = coeff_matrix[direction].eigenvects()
  

In [9]:
# generate boundary conditions. We either use the interior algorithm to update the

# characeteristic field or set the time derivative of the field to zero. Depends on
# if the mode is outgoing or incoming.

interior=Matrix([Bdotx, Bdoty, Bdotz, Ddotx, Ddoty, Ddotz, PsiBdot, PsiDdot])

time_derivs = Matrix([var('dt_{}'.format(func.name)) for func in funclist])

up_eqns = {}
dn_eqns = {}
up_boundary_conditions = {}
dn_boundary_conditions = {}

for direction in x, y, z:
    up_eqns[direction] = []
    dn_eqns[direction] = []

    for eig in eigen[direction]:
        val = eig[0].subs(PP *IMU,1)
        if val > 0:
            for j in range(eig[1]):
                # Setting time derivative of Eigenfunction to zero for up_boundary
                up_eqns[direction].append(Eq(time_derivs.dot(eig[2][j]),0))
                # setting time derivative of Eigenfunciton equal to interior algorithm for down_boundary
                dn_eqns[direction].append(Eq(time_derivs.dot(eig[2][j]),interior.dot(eig[2][j])))
        if (val < 0):
            for j in range(eig[1]):
                # setting time derivative of Eigenfunciton equal to interior algorithm for up_boundary
                up_eqns[direction].append(Eq(time_derivs.dot(eig[2][j]),interior.dot(eig[2][j])))
                # Setting time derivative of Eigenfunction to zero for down_boundary
                dn_eqns[direction].append(Eq(time_derivs.dot(eig[2][j]),0))

    up_boundary_conditions[direction] = solve(up_eqns[direction], time_derivs)
    dn_boundary_conditions[direction] = solve(dn_eqns[direction], time_derivs)


In [10]:
# final rule must be defined after all previous replacements
ijk = Idx('ijk', 1)
final_rule = {DX : IndexedBase('Dx')[ijk], DY : IndexedBase('Dy')[ijk], DZ : IndexedBase('Dz')[ijk],
              BX : IndexedBase('Bx')[ijk], BY : IndexedBase('By')[ijk], BZ : IndexedBase('Bz')[ijk],
              SIGMA : IndexedBase('sigma')[ijk], RHO : IndexedBase('rho')[ijk],
              PSIB : IndexedBase('PsiB')[ijk], PSID : IndexedBase('PsiD')[ijk],
              IMU : IndexedBase('imu')[ijk], IEPS : IndexedBase('ieps')[ijk]}

In [11]:
usedvar=set()
for ev in Bdotx, Bdoty, Bdotz, Ddotx, Ddoty, Ddotz, rhodot, PsiBdot, PsiDdot:
    usedvar = usedvar.union(ev.free_symbols)

In [12]:
diffvar = [ i for i in usedvar if re.match('^d[xyz]_',i.name[0:3])]
differentiated = sorted(
    list(
        set([ i.name[3:] for i in diffvar if re.match('^d[xyz]_',i.name) ])))

In [13]:
differentiated

['Bx', 'By', 'Bz', 'Dx', 'Dy', 'Dz', 'PsiB', 'PsiD', 'ieps', 'imu', 'sigma']

In [14]:
temp = sys.stdout
sys.stdout = output_header
# Print out Interior Algorithm
print('#define SIMPLE_MAXWELL_DERIVS ', end='')
for name in differentiated:
    print("\\")
    print("    const double dx_" + name + " = DIFFX(" + name +");\\")
    print("    const double dy_" + name + " = DIFFY(" + name +");\\")
    print("    const double dz_" + name + " = DIFFZ(" + name +");", end='')
print()
print()
print('#define SIMPLE_MAXWELL_INTERIOR_DOT \\')
print ('    SIMPLE_MAXWELL_DERIVS;\\')
print ("    ", IndexedBase('dotBx')[ijk],  " = ", ccode(Bdotx.subs(final_rule)), ";\\", sep='')
print ("    ", IndexedBase('dotBy')[ijk],  " = ", ccode(Bdoty.subs(final_rule)), ";\\", sep='')
print ("    ", IndexedBase('dotBz')[ijk],  " = ", ccode(Bdotz.subs(final_rule)), ";\\", sep='')
print('    \\')
print ("    ", IndexedBase('dotDx')[ijk],  " = ", ccode(Ddotx.subs(final_rule)), ";\\", sep='')
print ("    ", IndexedBase('dotDy')[ijk],  " = ", ccode(Ddoty.subs(final_rule)), ";\\", sep='')
print ("    ", IndexedBase('dotDz')[ijk],  " = ", ccode(Ddotz.subs(final_rule)), ";\\", sep='')
print ('    \\')
print ("    ", IndexedBase('dotPsiB')[ijk],  " = ", ccode(PsiBdot.subs(final_rule)), ";\\", sep='')
print ("    ", IndexedBase('dotPsiD')[ijk],  " = ", ccode(PsiDdot.subs(final_rule)), ";\\", sep='')
print("    ", IndexedBase('dotrho')[ijk],  " = ", ccode(rhodot.subs(final_rule)), ";", sep='')
print()
sys.stdout = temp

In [15]:
allbdry = [ value for dic in dn_boundary_conditions.values() for value in dic.values() ]

usedvar=set()
for ev in allbdry:
    usedvar = usedvar.union(ev.free_symbols)
    
diffvar = [ i for i in usedvar if re.match('^d[xyz]_',i.name[0:3])]
differentiated = sorted(
    list(
        set([ i.name[3:] for i in diffvar if re.match('^d[xyz]_',i.name) ])))

In [16]:
temp = sys.stdout
sys.stdout = output_header
print('#define SIMPLE_MAXWELL_BDERIVS ', end='')
for name in differentiated:
    print("\\")
    print("    const double dx_" + name + " = DIFFX(" + name +");\\")
    print("    const double dy_" + name + " = DIFFY(" + name +");\\")
    print("    const double dz_" + name + " = DIFFZ(" + name +");", end='')
print()
print()
sys.stdout = temp

In [17]:
temp = sys.stdout
sys.stdout = output_header
# Print out Boundary Algorithm

name = {x : "X", y : "Y", z : "Z"}
for direction in x, y, z:
    print ("#define SIMPLE_MAXWELL_LOW{0}_BOUNDARY \\".format(name[direction]))
    print('    SIMPLE_MAXWELL_BDERIVS;\\')
    print("    const double PP = sqrt(ieps[ijk]/imu[ijk]);\\")
    for key in time_derivs:
        print("    ", key.name.replace('dt_', 'dot'), "[ijk]", " = ",
              ccode(0.5 * 2 * dn_boundary_conditions[direction][key].subs(final_rule).expand()), ";\\", sep='')
    print("    dotrho[ijk] = 0.0;")

    print()
    print ("#define SIMPLE_MAXWELL_HIGH{0}_BOUNDARY \\".format(name[direction]))
    print ('    SIMPLE_MAXWELL_BDERIVS;\\')
    print ("    const double PP = sqrt(ieps[ijk]/imu[ijk]);\\")
    for key in time_derivs:
        print("    ", key.name.replace('dt_', 'dot'),"[ijk]", " = ",
              ccode(0.5 * 2 * up_boundary_conditions[direction][key].subs(final_rule).expand()), ";\\", sep='')
    print("    dotrho[ijk] = 0.0;")

    print()

sys.stdout = temp

In [18]:
output_header.close()

In [19]:
!clang-format simple_maxwell.h > SimpleMaxwell.h